In [1]:
import inspect

from mintalib.core import calc_sma
from mintalib.samples import sample_prices
from mintalib.builder import annotate_parameter

In [2]:
prices = sample_prices()
prices


,open,high,low,close,volume
date,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600
...,...,...,...,...,...
2024-10-15,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17,233.429993,233.850006,230.520004,232.149994,32993800


In [3]:
signature = inspect.signature(calc_sma)
print(signature)
parameters = signature.parameters
parameters

(series, period, *, wrap=False)


mappingproxy({'series': <Parameter "series">,
              'period': <Parameter "period">,
              'wrap': <Parameter "wrap=False">})

In [4]:
calc_sma.__annotations__


{}

In [ ]:
def make_signature(calc_func):
    """creates function signature from core function"""
    sig = inspect.signature(calc_func)
    first_param = next(iter(sig.parameters.values()))
    
    new_params = []
    for param in sig.parameters.values():
        if param.name == "series":
            param = param.replace(name="prices")
        if param.name == "wrap":
            continue
        param = annotate_parameter(param)
        new_params.append(param)

    if first_param.name == "series":
        item_param = inspect.Parameter(
            name="item",
            kind=inspect.Parameter.KEYWORD_ONLY,
            default=None,
            annotation=str
        )
        new_params.append(item_param)

    return sig.replace(parameters=new_params)

sig = make_signature(calc_sma)
print(sig)

(prices, period: int, *, item: str = None)


In [6]:
from functools import wraps
from mintalib.core import get_series, column_accessor, wrap_result

def wrap_function(calc_func):
    signature = inspect.signature(calc_func)
    first_param = next(iter(signature.parameters))
    def decorator(func):
        @wraps(func)
        def wrapper(prices, *args, **kwargs):
            item = kwargs.pop('item', None)

            if first_param == 'series':
                input = get_series(prices, item)
            else:
                input = column_accessor(prices)

            print("call", input, args, kwargs)
            result = calc_func(input, *args, **kwargs)
            return wrap_result(result, prices)
        return wrapper
    return decorator

In [7]:
@wrap_function(calc_sma)
def SMA(series, *, period: int, item: str = None): ...


In [8]:
inspect.signature(SMA)


<Signature (series, *, period: int, item: str = None)>

In [9]:


calc_sma(prices.close, 5)

array([         nan,          nan,          nan, ..., 231.32600098,
       232.81600037, 233.8519989 ])

In [10]:
SMA(prices, period=5, item='close')

call date
1980-12-12      0.098943
1980-12-15      0.093781
1980-12-16      0.086898
1980-12-17      0.089049
1980-12-18      0.091630
                 ...    
2024-10-15    233.850006
2024-10-16    231.779999
2024-10-17    232.149994
2024-10-18    235.000000
2024-10-21    236.479996
Name: close, Length: 11056, dtype: float64 () {'period': 5}


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18      0.092060
                 ...    
2024-10-15    230.256000
2024-10-16    230.704001
2024-10-17    231.326001
2024-10-18    232.816000
2024-10-21    233.851999
Length: 11056, dtype: float64

In [11]:
def make_function(calc_func):
    signature = inspect.signature(calc_func)
    cname = f"core.{calc_func.__name__}"
    fname = calc_func.__name__.removeprefix("calc_").upper()
    signature = make_signature(calc_func)
    buffer = f"@wrap_function({cname})\n"
    buffer += f"def {fname}{signature}: ...\n\n"
    print(buffer)

make_function(calc_sma)

@wrap_function(core.calc_sma)
def SMA(prices, period: int, *, item: str = None): ...


